## FTP Connection

### Connection Parameters

In [3]:
server = "opendata.dwd.de"
user   = "anonymous"
passwd = ""

### FTP Directory Definition and Station Description Filename Pattern

In [4]:
# The topic of interest.
topic_dir = "/monthly/more_precip/recent/"
#topic_dir = "/annual/kl/historical/"

# This is the search pattern common to ALL station description file names 
station_desc_pattern = "_Beschreibung_Stationen.txt"

# Below this directory tree node all climate data are stored.
ftp_climate_data_dir = "/climate_environment/CDC/observations_germany/climate/"
ftp_dir =  ftp_climate_data_dir + topic_dir

### Local Directories

In [5]:
local_ftp_dir         = "../data_4/original/DWD/"      # Local directory to store local ftp data copies, the local data source or input data. 
local_ftp_station_dir = local_ftp_dir + topic_dir # Local directory where local station info is located
local_ftp_ts_dir      = local_ftp_dir + topic_dir # Local directory where time series downloaded from ftp are located

local_generated_dir   = "../data_4/generated/DWD/" # The generated of derived data in contrast to local_ftp_dir
local_station_dir     = local_generated_dir + topic_dir # Derived station data, i.e. the CSV file
local_ts_merged_dir   = local_generated_dir + topic_dir # Parallelly merged time series, wide data frame with one TS per column
local_ts_appended_dir = local_generated_dir + topic_dir # Serially appended time series, long data frame for QGIS TimeManager Plugin_

In [6]:
print(local_ftp_dir)
print(local_ftp_station_dir)
print(local_ftp_ts_dir)
print()
print(local_generated_dir)
print(local_station_dir)
print(local_ts_merged_dir)
print(local_ts_appended_dir)

../data_4/original/DWD/
../data_4/original/DWD//monthly/more_precip/recent/
../data_4/original/DWD//monthly/more_precip/recent/

../data_4/generated/DWD/
../data_4/generated/DWD//monthly/more_precip/recent/
../data_4/generated/DWD//monthly/more_precip/recent/
../data_4/generated/DWD//monthly/more_precip/recent/


In [7]:
import os
os.makedirs(local_ftp_dir,exist_ok = True) # it does not complain if the dir already exists.
os.makedirs(local_ftp_station_dir,exist_ok = True)
os.makedirs(local_ftp_ts_dir,exist_ok = True)

os.makedirs(local_generated_dir,exist_ok = True)
os.makedirs(local_station_dir,exist_ok = True)
os.makedirs(local_ts_merged_dir,exist_ok = True)
os.makedirs(local_ts_appended_dir,exist_ok = True)

### FTP Connect


In [8]:
import ftplib
ftp = ftplib.FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)

230 Login successful.


In [9]:
ret = ftp.cwd(".")

### FTP Grab File Function

In [10]:
def grabFile(ftpfullname,localfullname):
    try:
        ret = ftp.cwd(".") # A dummy action to chack the connection and to provoke an exception if necessary.
        localfile = open(localfullname, 'wb')
        ftp.retrbinary('RETR ' + ftpfullname, localfile.write, 1024)
        localfile.close()
    
    except ftplib.error_perm:
        print("FTP ERROR. Operation not permitted. File not found?")

    except ftplib.error_temp:
        print("FTP ERROR. Timeout.")

    except ConnectionAbortedError:
        print("FTP ERROR. Connection aborted.")



### Generate Pandas Dataframe from FTP Directory Listing

In [11]:
import pandas as pd
import os

def gen_df_from_ftp_dir_listing(ftp, ftpdir):
    lines = []
    flist = []
    try:    
        res = ftp.retrlines("LIST "+ftpdir, lines.append)
    except:
        print("Error: ftp.retrlines() failed. ftp timeout? Reconnect!")
        return
        
    if len(lines) == 0:
        print("Error: ftp dir is empty")
        return
    
    for line in lines:
#        print(line)
        [ftype, fsize, fname] = [line[0:1], int(line[31:42]), line[56:]]
#        itemlist = [line[0:1], int(line[31:42]), line[56:]]
#        flist.append(itemlist)
        
        fext = os.path.splitext(fname)[-1]
        
        if fext == ".zip":
            station_id = int(fname.split("_")[2])
        else:
            station_id = -1 
        
        flist.append([station_id, fname, fext, fsize, ftype])
        
        

    df_ftpdir = pd.DataFrame(flist,columns=["station_id", "name", "ext", "size", "type"])
    return(df_ftpdir)

In [12]:
df_ftpdir = gen_df_from_ftp_dir_listing(ftp, ftp_dir)

In [13]:
df_ftpdir.head(10)

,station_id,name,ext,size,type
0,-1,RR_Monatswerte_Beschreibung_Stationen.txt,.txt,1149389,-
1,6,monatswerte_RR_00006_akt.zip,.zip,6760,-
2,15,monatswerte_RR_00015_akt.zip,.zip,6601,-
3,19,monatswerte_RR_00019_akt.zip,.zip,7066,-
4,20,monatswerte_RR_00020_akt.zip,.zip,9290,-
5,21,monatswerte_RR_00021_akt.zip,.zip,8239,-
6,22,monatswerte_RR_00022_akt.zip,.zip,6913,-
7,23,monatswerte_RR_00023_akt.zip,.zip,6855,-
8,41,monatswerte_RR_00041_akt.zip,.zip,7433,-
9,44,monatswerte_RR_00044_akt.zip,.zip,8925,-


### Dataframe with TS Zip Files

In [14]:
#df_ftpdir["ext"]==".zip"
df_zips = df_ftpdir[df_ftpdir["ext"]==".zip"]
df_zips.set_index("station_id", inplace = True)
df_zips.head(10)

,name,ext,size,type
station_id,,,,
6,monatswerte_RR_00006_akt.zip,.zip,6760,-
15,monatswerte_RR_00015_akt.zip,.zip,6601,-
19,monatswerte_RR_00019_akt.zip,.zip,7066,-
20,monatswerte_RR_00020_akt.zip,.zip,9290,-
21,monatswerte_RR_00021_akt.zip,.zip,8239,-
22,monatswerte_RR_00022_akt.zip,.zip,6913,-
23,monatswerte_RR_00023_akt.zip,.zip,6855,-
41,monatswerte_RR_00041_akt.zip,.zip,7433,-
44,monatswerte_RR_00044_akt.zip,.zip,8925,-


### Download the Station Description File

In [15]:
station_fname = df_ftpdir[df_ftpdir['name'].str.contains(station_desc_pattern)]["name"].values[0]
print(station_fname)

# ALternative
#station_fname2 = df_ftpdir[df_ftpdir["name"].str.match("^.*Beschreibung_Stationen.*txt$")]["name"].values[0]
#print(station_fname2)

RR_Monatswerte_Beschreibung_Stationen.txt


In [16]:
print("grabFile: ")
print("From: " + ftp_dir + station_fname)
print("To:   " + local_ftp_station_dir + station_fname)
grabFile(ftp_dir + station_fname, local_ftp_station_dir + station_fname)

grabFile: 
From: /climate_environment/CDC/observations_germany/climate//monthly/more_precip/recent/RR_Monatswerte_Beschreibung_Stationen.txt
To:   ../data_4/original/DWD//monthly/more_precip/recent/RR_Monatswerte_Beschreibung_Stationen.txt


In [17]:
# extract column names. They are in German (de)
# We have to use codecs because of difficulties with character encoding (German Umlaute)
import codecs

def station_desc_txt_to_csv(txtfile, csvfile):
    file = codecs.open(txtfile,"r",encoding= "cp1252")
    r = file.readline()
    file.close()
    colnames_de = r.split()
    colnames_de
    
    translate = \
    {'Stations_id':'station_id',
     'von_datum':'date_from',
     'bis_datum':'date_to',
     'Stationshoehe':'altitude',
     'geoBreite': 'latitude',
     'geoLaenge': 'longitude',
     'Stationsname':'name',
     'Bundesland':'state'}
    
    colnames_en = [translate[h] for h in colnames_de]
    
    # Skip the first two rows and set the column names.
    df = pd.read_fwf(txtfile,skiprows=2,names=colnames_en, parse_dates=["date_from","date_to"],index_col = 0, encoding="cp1252")
    
    # write csv
    df.to_csv(csvfile, sep = ";")
    return(df)

In [18]:
basename = os.path.splitext(station_fname)[0]
df_stations = station_desc_txt_to_csv(local_ftp_station_dir + station_fname, local_station_dir + basename + ".csv")
df_stations.head()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
1,1891-01-01,1986-06-30,478,47.8413,8.8493,Aach,Baden-Württemberg
2,1914-01-01,2006-12-31,138,50.8066,6.0996,Aachen (Kläranlage),Nordrhein-Westfalen
3,1844-01-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
4,1898-01-01,1979-10-31,243,50.7683,6.1207,Aachen-Brand,Nordrhein-Westfalen
6,1982-11-01,2023-02-28,455,48.8361,10.0598,Aalen-Unterrombach,Baden-Württemberg


### Select Stations Located in NRW from Station Description Dataframe

In [19]:
station_ids_selected = df_stations[df_stations['state'].str.contains("Nordrhein")].index
station_ids_selected

Int64Index([    2,     3,     4,    38,    42,    79,    98,   104,   110,
              186,
            ...
            16087, 19042, 19043, 19125, 19306, 19309, 19331, 19414, 19415,
            19424],
           dtype='int64', name='station_id', length=436)

In [20]:
# Create variable with TRUE if state is Nordrhein-Westfalen

# isNRW = df_stations['state'] == "Nordrhein-Westfalen"
isNRW = df_stations['state'].str.contains("Nordrhein")

# Create variable with TRUE if date_to is latest date (indicates operation up to now)
isOperational = df_stations['date_to'] == df_stations.date_to.max() 

#isBefore1950 = df_stations['date_from'] < '1950'
#dfNRW = df_stations[isNRW & isOperational & isBefore1950]

# select on both conditions

dfNRW = df_stations[isNRW & isOperational]

#print("Number of stations in NRW: \n", dfNRW.count())
dfNRW

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
110,1891-01-01,2023-02-28,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen
216,1945-01-01,2023-02-28,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
325,1900-01-01,2023-02-28,127,51.7948,8.0718,Beckum-Vellern,Nordrhein-Westfalen
389,1891-01-01,2023-02-28,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen
390,1986-12-01,2023-02-28,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
...,...,...,...,...,...,...,...
19306,2021-06-01,2023-02-28,138,52.0249,9.0454,Dörentrup-Humfeld,Nordrhein-Westfalen
19309,2021-06-01,2023-02-28,20,51.6369,6.2865,Kevelaer-Kervenheim,Nordrhein-Westfalen
19414,2022-04-01,2023-02-28,51,52.1890,7.1962,Ochtrup (Süd),Nordrhein-Westfalen


### Download TS Data from FTP Server

Problem: Not all stations listed in the station description file are associated with a time series (zip file)! The stations in the description file and the set of stations whoch are TS data provided for (zip files) do not match perfectly.  

In [21]:
# Add the names of the zip files only to a list. 
local_zip_list = []

station_ids_selected = list(dfNRW.index)

for station_id in station_ids_selected:
    try:
        fname = df_zips["name"][station_id]
        print(fname)
        grabFile(ftp_dir + fname, local_ftp_ts_dir + fname)
        local_zip_list.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

monatswerte_RR_00110_akt.zip
monatswerte_RR_00216_akt.zip
monatswerte_RR_00325_akt.zip
monatswerte_RR_00389_akt.zip
monatswerte_RR_00390_akt.zip
monatswerte_RR_00488_akt.zip
monatswerte_RR_00533_akt.zip
monatswerte_RR_00554_akt.zip
monatswerte_RR_00555_akt.zip
monatswerte_RR_00603_akt.zip
monatswerte_RR_00613_akt.zip
monatswerte_RR_00617_akt.zip
monatswerte_RR_00644_akt.zip
monatswerte_RR_00796_akt.zip
monatswerte_RR_00871_akt.zip
monatswerte_RR_00902_akt.zip
monatswerte_RR_00934_akt.zip
monatswerte_RR_00989_akt.zip
monatswerte_RR_01024_akt.zip
monatswerte_RR_01046_akt.zip
monatswerte_RR_01054_akt.zip
monatswerte_RR_01078_akt.zip
monatswerte_RR_01232_akt.zip
monatswerte_RR_01241_akt.zip
monatswerte_RR_01246_akt.zip
monatswerte_RR_01277_akt.zip
monatswerte_RR_01283_akt.zip
monatswerte_RR_01298_akt.zip
monatswerte_RR_01300_akt.zip
monatswerte_RR_01303_akt.zip
monatswerte_RR_01327_akt.zip
monatswerte_RR_01363_akt.zip
monatswerte_RR_01590_akt.zip
monatswerte_RR_01595_akt.zip
monatswerte_RR

### Join (Merge) the Time Series Columns

In [22]:
from datetime import datetime

def prec_ts_to_df(fname):
    
    dateparse = lambda dates: [datetime.strptime(str(d), '%Y%m%d') for d in dates]

    df = pd.read_csv(fname, delimiter=";", encoding="utf8", index_col="MESS_DATUM_BEGINN", parse_dates = ["MESS_DATUM_BEGINN"], date_parser = dateparse, na_values = [-999.0, -999])

    #df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2",\
    #             index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse)
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.

    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_', regex=False).str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    df.index.name = df.index.name.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
    return(df)

In [23]:
from zipfile import ZipFile


def prec_ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = prec_ts_to_df(myfile)
                #s = dftmp["mo_rr"].rename(dftmp["stations_id"][0]).to_frame()
                s = dftmp["mo_rr"].rename("ID_" + str(dftmp["stations_id"][0])).to_frame()
                # outer merge.
                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
    df.index.rename(name = "month", inplace = True)
    return(df)

In [24]:
df_merged_ts = prec_ts_merge()

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00110_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00110.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00216_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00216.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00325_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00325.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00389_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00389.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00390_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00390.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00488_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00488.txt

Zip archiv

In [25]:
df_merged_ts.head()

,ID_110,ID_216,ID_325,ID_389,ID_390,ID_488,ID_533,ID_554,ID_555,ID_603,...,ID_13700,ID_13713,ID_15000,ID_15455,ID_19125,ID_19306,ID_19309,ID_19414,ID_19415,ID_19424
month,,,,,,,,,,,,,,,,,,,,,
2021-08-01,41.2,92.8,71.1,107.2,94.9,96.8,78.1,43.5,NaN,115.9,...,85.8,128.3,95.8,66.5,58.8,111.5,61.4,NaN,NaN,NaN
2021-09-01,34.1,37.1,17.6,42.7,48.6,42.8,22.3,46.3,NaN,18.2,...,30.6,65.5,18.9,13.9,38.0,36.5,47.8,NaN,NaN,NaN
2021-10-01,53.5,74.9,45.3,58.5,54.4,53.9,31.1,64.9,NaN,49.5,...,100.6,110.4,57.8,36.1,50.7,50.6,72.1,NaN,NaN,NaN
2021-11-01,30.8,55.6,31.9,49.9,46.3,49.4,35.1,41.1,NaN,46.7,...,65.6,85.6,41.0,27.9,24.1,48.6,42.3,NaN,NaN,NaN
2021-12-01,46.3,83.9,45.5,64.2,69.8,68.6,47.3,46.6,NaN,51.6,...,73.6,126.4,73.7,54.4,44.9,66.6,49.0,NaN,NaN,NaN


In [26]:
# prec or temp?
filepathname = local_ts_merged_dir + "prec_ts_merged.csv"
print("df_merged_ts is saved to: %s" % (filepathname))
df_merged_ts.to_csv(filepathname,sep=";")

df_merged_ts is saved to: ../data_4/generated/DWD//monthly/more_precip/recent/prec_ts_merged.csv


In [27]:
def ts_append():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
# TEMPERATURE                dftmp = temp_ts_to_df(myfile)
# PRECIPIATION
                dftmp = prec_ts_to_df(myfile)
                dftmp = dftmp.merge(df_stations,how="inner",left_on="stations_id",right_on="station_id",right_index=True)
#                print(dftmp.head(5))
                df = df.append(dftmp)

    #df.index.names = ["year"]
    #df.index.rename(name = "time", inplace = True)
    return(df)

In [28]:
df_appended_ts = ts_append()

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00110_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00110.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00216_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00216.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00325_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00325.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00389_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00389.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00390_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00390.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00488_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00488.txt

Zip archiv

C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append 

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00617_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00617.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00644_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00644.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00796_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00796.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00871_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00871.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00902_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00902.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_00934_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_00934.txt

Zip archiv

C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append 

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_01046_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_01046.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_01054_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_01054.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_01078_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_01078.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_01232_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_01232.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_01241_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_01241.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_01246_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_01246.txt



...

C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append 

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_07374_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_07374.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_07378_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_07378.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_13670_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_13670.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_13671_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_13671.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_13696_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_13696.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_13700_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_13700.txt

Zip archiv

C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append 

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_19309_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_19309.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_19414_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_19414.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_19415_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_19415.txt

Zip archive: ../data_4/original/DWD//monthly/more_precip/recent/monatswerte_RR_19424_akt.zip
Extract product file: produkt_nieder_monat_20210801_20230228_19424.txt



C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
C:\Users\meghn\AppData\Local\Temp\ipykernel_31324\2389752127.py:18: FutureWarning: The frame.append 

In [29]:
filepathname = local_ts_appended_dir + "prec_ts_appended.csv"
print("df_appended_ts saved to: %s" % (filepathname))
df_appended_ts.to_csv(filepathname,sep=";")

df_appended_ts saved to: ../data_4/generated/DWD//monthly/more_precip/recent/prec_ts_appended.csv


In [30]:
df_appended_ts.head()

,stations_id,mess_datum_ende,qn_6,mo_nsh,mo_rr,mo_sh_s,mx_rs,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum_beginn,,,,,,,,,,,,,,,
2021-08-01,110,20210831,9.0,0.0,41.2,0.0,7.4,eor,1891-01-01,2023-02-28,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen
2021-09-01,110,20210930,3.0,0.0,34.1,0.0,17.7,eor,1891-01-01,2023-02-28,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen
2021-10-01,110,20211031,3.0,0.0,53.5,0.0,9.6,eor,1891-01-01,2023-02-28,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen
2021-11-01,110,20211130,3.0,0.0,30.8,0.0,10.1,eor,1891-01-01,2023-02-28,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen
2021-12-01,110,20211231,3.0,0.0,46.3,0.0,7.3,eor,1891-01-01,2023-02-28,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen
